In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
item=pd.read_csv('item_data.csv')
log=pd.read_csv('log_items.csv')

In [4]:
y=train['is_click']
train.drop('is_click',axis=1,inplace=True)

In [5]:
df=pd.concat([train,test],axis=0,sort=False)

In [6]:
df.columns=['impression_id', 'server_time', 'user_id', 'app_code', 'os_version',
       'is_4G']

In [7]:
log.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
server_time=np.array(log['server_time'])
for i in range(server_time.shape[0]):
    time=int(server_time[i][9:11])
    server_time[i]=time
log['server_time']=pd.Series(server_time)

In [9]:
conditions=[
    ((log['server_time']) >=0) & ((log['server_time']) <6),
    ((log['server_time']) >=6) & ((log['server_time']) <9),
    ((log['server_time']) >=9) & ((log['server_time']) <12),
    ((log['server_time']) >=12) & ((log['server_time']) <16),
    ((log['server_time']) >=16) & ((log['server_time']) <20),
    ((log['server_time']) >=20) & ((log['server_time']) <24),
]
choices = ['midnight','early_morning','morning','afternoon','evening','night']
log['time']=np.select(conditions,choices,default='night')
log.drop('server_time',axis=1,inplace=True)

In [10]:
df.head(2)

,impression_id,server_time,user_id,app_code,os_version,is_4G
0,c4ca4238a0b923820dcc509a6f75849b,15-11-18 00:00,87862,422,old,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,15-11-18 00:01,63410,467,latest,1


In [11]:
server_time=np.array(df['server_time'])
for i in range(server_time.shape[0]):
    time=int(server_time[i][9:11])
    server_time[i]=time
df['server_time']=pd.Series(server_time)
conditions=[
    ((df['server_time']) >=0) & ((df['server_time']) <6),
    ((df['server_time']) >=6) & ((df['server_time']) <9),
    ((df['server_time']) >=9) & ((df['server_time']) <12),
    ((df['server_time']) >=12) & ((df['server_time']) <16),
    ((df['server_time']) >=16) & ((df['server_time']) <20),
    ((df['server_time']) >=20) & ((df['server_time']) <24),
]
choices = ['midnight','early_morning','morning','afternoon','evening','night']
df['time']=np.select(conditions,choices,default='night')
df.drop('server_time',axis=1,inplace=True)

In [12]:
%%time
string='item_id'
df[string]=df[['user_id','time']].merge(log,how='left').item_id
df[string].fillna(-1, inplace=True)

l=np.array([425.0,76404.0,79527.0 ,68968.0 ,64835.0,119516.0,27574.0,3500.0,84704.0,30152.0,49461.0,127900.0,11871.0,30048.0,113905.0,72120.0,76156.0,61487.0,58320.0,88587.0,126140.0,54546.0,72955.0,33633.0,38922.0,73972.0,85108.0,1528.0,8365.0])


item_id=np.array(df['item_id'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['item_id']=pd.Series(item_id)

Wall time: 1.61 s


In [13]:
%%time
string='device_type'
df[string]=df[['user_id','time']].merge(log,how='left').device_type
df[string].fillna(df[string].mode()[0], inplace=True)

Wall time: 1.25 s


In [14]:
%%time
string='session_id'
df[string]=df[['user_id','time']].merge(log,how='left').session_id
df[string].fillna(-1, inplace=True)
l=np.array([886941.0,832824.0,676414.0,632525.0,1047176.0,600140.0,218759.0,268638.0,115450.0,941339.0,146111.0,609165.0,30559.0,749940.0,548210.0,458847.0,628148.0,373409.0,971160.0,144182.0,484416.0,375924.0,507005.0,606519.0,325829.0,947011.0,96485.0 ,842147.0,7883.0,850779.0])


item_id=np.array(df['session_id'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['session_id']=pd.Series(item_id)

Wall time: 1.51 s


In [15]:
df.head(10)

,impression_id,user_id,app_code,os_version,is_4G,time,item_id,device_type,session_id
0,c4ca4238a0b923820dcc509a6f75849b,87862,422,old,0,midnight,425.0,android,886941.0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,63410,467,latest,1,midnight,43209.0,android,343558.0
2,70efdf2ec9b086079795c442636b55fb,71748,259,intermediate,1,midnight,43209.0,android,343558.0
3,8e296a067a37563370ded05f5a3bf3ec,69209,244,latest,1,midnight,76404.0,android,832824.0
4,182be0c5cdcd5072bb1864cdee4d3d6e,62873,473,latest,0,midnight,79527.0,android,676414.0
5,3416a75f4cea9109507cacd8e2f2aefc,67352,409,latest,1,midnight,68968.0,android,632525.0
6,f457c545a9ded88f18ecee47145a72c0,64356,190,intermediate,0,midnight,64835.0,android,1047176.0
7,72b32a1f754ba1c09b3695e0cb6cde7f,27329,481,latest,0,midnight,8160.0,android,7621.0
8,fc490ca45c00b1249bbe3554a4fdf6fb,83318,386,old,0,midnight,63026.0,android,7621.0
9,d2ddea18f00665ce8623e36bd4e3c7c5,70206,190,latest,0,midnight,63026.0,android,7621.0


In [16]:
log.head(1)

,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type,time
0,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0,early_morning


In [17]:
%%time
string='item_price'
df[string]=df[['user_id','time']].merge(log,how='left').item_price
df[string].fillna(-1, inplace=True)
l=np.array([6336.0,2496.0,633.0 ,1856.0,2547.0,1907.0,2227.0,3833.0,1025.0,1145.0,8960.0,627.0 ,3187.0,889.0 ,3225.0,691.0 ,6841.0,5178.0,2560.0,1024.0,576.0 ,2304.0,2553.0,1075.0,2880.0,3136.0,768.0 ,4416.0,1152.0,1728.0])


item_id=np.array(df['item_price'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['item_price']=pd.Series(item_id)

Wall time: 1.58 s


In [18]:
%%time
string='category_1'
df[string]=df[['user_id','time']].merge(log,how='left').category_1
df[string].fillna(-1, inplace=True)
l=np.array([9.0,17.0,1.0 ,4.0 ,11.0,13.0,10.0,7.0 ,8.0 ,14.0,12.0,0.0 ,16.0,15.0,6.0])



item_id=np.array(df['category_1'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['category_1']=pd.Series(item_id)

Wall time: 1.51 s


In [19]:
%%time
string='category_2'
df[string]=df[['user_id','time']].merge(log,how='left').category_2
df[string].fillna(-1, inplace=True)
l=np.array([38.0,8.0,57.0,39.0,43.0,3.0,42.0,9.0 ,51.0,44.0,74.0,58.0,63.0,40.0,45.0,28.0,62.0 ,7.0 ,41.0,26.0,24.0,67.0,6.0 ,61.0,37.0,35.0,36.0,29.0,76.0,56.0])

item_id=np.array(df['category_2'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['category_2']=pd.Series(item_id)

Wall time: 1.61 s


In [20]:
%%time
string='category_3'
df[string]=df[['user_id','time']].merge(log,how='left').category_3
df[string].fillna(-1, inplace=True)
l=np.array([62.0,292.,114.0,223.0 ,279.0,157.0,36.0,159.0 ,84.0,244.0,151.0,252.0,100.0,113.0,90.0,220.0,162.0,263.0,73.0,264.0,111.0,203.0,27.0 ,44.0,147.0,233.0 ,283.0,171.0,301.0,172.0])


item_id=np.array(df['category_3'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['category_3']=pd.Series(item_id)

Wall time: 1.59 s


In [21]:
%%time
string='product_type'
df[string]=df[['user_id','time']].merge(log,how='left').product_type
df[string].fillna(-1, inplace=True)
l=np.array([2874.0,5036.0,10412.0,2106.0,7455.0,5622.0,4524.0,10025.0,6239.0,7791.0,3055.0,5276.0,5251.0,2099.0,8121.0,5164.0,10083.0,1485.0,3928.0,9959.0,10099.0,1923.0,10392.0,1617.0,5168.0,2417.0,10449.0,5609.0,9041.0,907.0])



item_id=np.array(df['product_type'])
con=item_id[0]
k=0
for i in range(item_id.shape[0]):
    if item_id[i]==con:
        item_id[i]=l[k%l.shape[0]]
        k+=1
df['product_type']=pd.Series(item_id)

Wall time: 1.48 s


In [22]:
df.head(2)

,impression_id,user_id,app_code,os_version,is_4G,time,item_id,device_type,session_id,item_price,category_1,category_2,category_3,product_type
0,c4ca4238a0b923820dcc509a6f75849b,87862,422,old,0,midnight,425.0,android,886941.0,6336.0,9.0,38.0,62.0,2874.0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,63410,467,latest,1,midnight,43209.0,android,343558.0,3421.0,4.0,74.0,292.0,577.0


In [23]:
df.device_type.value_counts(dropna=False).head()

android    328169
iphone        105
web            10
Name: device_type, dtype: int64

In [24]:
# df = df[np.isfinite(df['product_type'])]

In [25]:
df.head(2)

,impression_id,user_id,app_code,os_version,is_4G,time,item_id,device_type,session_id,item_price,category_1,category_2,category_3,product_type
0,c4ca4238a0b923820dcc509a6f75849b,87862,422,old,0,midnight,425.0,android,886941.0,6336.0,9.0,38.0,62.0,2874.0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,63410,467,latest,1,midnight,43209.0,android,343558.0,3421.0,4.0,74.0,292.0,577.0


In [26]:
x_train=df[:train.shape[0]]
x_test=df[train.shape[0]:]
x_train['is_click']=y
x_train.to_csv('Xx_train.csv',index=False)
x_test.to_csv('Xx_test.csv',index=False)

c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
